In [3]:
import kagglehub

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 42.6M/42.6M [00:01<00:00, 25.3MB/s]

Extracting files...


Path to dataset files: /home/jovyan/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2


# Read CSV 

Read csv from data folder and create pandas dataframe.

In [4]:
from __future__ import annotations

In [9]:
import pandas as pd
import numpy as np
from faker import Faker

In [10]:
import os
import uuid
import random
from dataclasses import dataclass
from datetime import date, datetime, timedelta
from typing import Dict, List, Tuple, Optional

In [11]:

fake = Faker()
Faker.seed(7)
random.seed(7)
np.random.seed(7)

In [197]:
@dataclass
class GenConfig:
    olist_dir: str
    out_dir: str = "output"
    start_date: date = date(2019,1,1)
    num_days: int = 15

    # daily change rate (dims)
    customer_change_rate: float = 0.01
    customer_delete_rate: float = 0.001
    customer_insert_rate: float = 0.002

    product_change_rate: float = 0.005
    product_delete_rate: float = 0.0005
    product_insert_rate: float = 0.001

    #daily fact generation scale
    new_orders_per_day: int = 1500
    return_rate: float = 0.03
    cancel_rate: float = 0.01

    # inventory behaviour
    restock_prob_per_product_per_day: float = 0.0008
    restock_qty_min: int = 20
    restock_qty_max: int = 300

    file_format: str = "csv"
    
    
    

In [198]:
cfg = GenConfig(
        olist_dir=r"data",
    )

In [24]:
def load_olist(cfg: GenConfig) -> Dict[str: pd.DataFrame]:
    d = {}
    base = cfg.olist_dir

    def r(name: str) -> pd.DataFrame:
        return pd.read_csv(os.path.join(base, name))

    d["customer"] = r("olist_customers_dataset.csv")
    d["orders"] = r("olist_orders_dataset.csv")
    d["order_items"] = r("olist_order_items_dataset.csv")
    d["products"] = r("olist_products_dataset.csv")
    d["payments"] = r("olist_order_payments_dataset.csv")
    d["reviews"] = r("olist_order_reviews_dataset.csv")
    d["sellers"] = r("olist_sellers_dataset.csv")
    d["geolocation"] = r("olist_geolocation_dataset.csv")
    d["category_translation"] = r("product_category_name_translation.csv")

    # Normalize timestamps
    for col in [
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ]:
        if col in d["orders"].columns:
            d["orders"][col] = pd.to_datetime(d["orders"][col], errors="coerce")

    return d
        

In [25]:
olist_data = load_olist(cfg)

# Create tables data
* customers
* customer_addresses
* customer_accounts
* products
* product_categories
* suppliers
* inventory
* orders
* order_items
* payments
* shipments
* returns

Customers, products, and suppliers are baseline tables.
first create them

In [30]:
olist_data["customer"]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [32]:
customers = olist_data["customer"].copy()
customers["created_ts"] = pd.Timestamp("2017-01-01")
customers["last_updated_ts"] = pd.Timestamp("2017-01-01")

In [33]:
products = olist_data["products"].copy()
products["created_ts"] = pd.Timestamp("2017-01-01")
products["last_updated_ts"] = pd.Timestamp("2017-01-01")

In [54]:
def build_suppliers_from_sellers(sellers: pd.DataFrame) -> pd.DataFrame:
    sup = sellers.copy()
    sup = sup.rename(columns={
        "seller_id": "supplier_id",
        "seller_zip_code_prefix": "zip_code_prefix",
        "seller_city": "city",
        "seller_state": "state" 
    })
    sup["supplier_status"] = "ACTIVE"
    sup["created_ts"] = pd.Timestamp("2017-01-01")
    sup["last_updated_ts"] = pd.Timestamp("2017-01-01")
    return sup[["supplier_id", "zip_code_prefix", "city", "state", "supplier_status", "created_ts", "last_updated_ts"]]

In [56]:
suppliers = build_suppliers_from_sellers(olist_data["sellers"])

In [77]:
def build_customer_addresses(customers: pd.DataFrame) -> pd.DataFrame:
    addr = customers[["customer_id", "customer_zip_code_prefix", "customer_city", "customer_state"]].copy()
    addr = addr.rename(columns = {
        "customer_zip_code_prefix": "zip_code_prefix",
        "customer_city": "city",
        "customer_state": "state"
    })
    addr["address_id"] = [str(uuid.uuid4()) for _ in range(len(addr))]
    addr["address_line1"] = [fake.street_address() for _ in range(len(addr))]
    addr["address_line2"] = [fake.secondary_address() for _ in range(len(addr))]
    addr["created_ts"] = pd.Timestamp("2017-01-01")
    addr["last_updated_ts"] = pd.Timestamp("2017-01-01")
    return addr[["address_id","customer_id", "address_line1", "address_line2", "zip_code_prefix",
                 "city", "state", "created_ts", "last_updated_ts"]]

In [78]:
customer_addresses = build_customer_addresses(customers)

In [79]:
customer_addresses

,address_id,customer_id,address_line1,address_line2,zip_code_prefix,city,state,created_ts,last_updated_ts
0,3f9c1f0f-2a0e-4ba6-a744-79f53a02f6ea,06b8999e2fba1a1fbc88172c00ba8bc7,37328 Johnson Lake Apt. 884,Suite 709,14409,franca,SP,2017-01-01,2017-01-01
1,fa9debac-518d-4ecb-8f20-bac32c7c80cb,18955e83d337fd6b2def6b18a428ac77,949 Duran Ville Apt. 791,Suite 964,9790,sao bernardo do campo,SP,2017-01-01,2017-01-01
2,8b17e525-e3e9-41fe-9228-9890c85f22cb,4e7b3e00288586ebd08712fdd0374a03,49652 Kelly Summit,Suite 667,1151,sao paulo,SP,2017-01-01,2017-01-01
3,9070f0e5-038a-433b-96c1-b4dc3d7f7b35,b2b6027bc5c5109e529d4dc6358b12c3,54512 Nancy Station,Apt. 331,8775,mogi das cruzes,SP,2017-01-01,2017-01-01
4,2cb4d314-280f-4b37-89f9-3c07001a10a6,4f2d8ab171c80ec8364f7c12e35b23ad,4391 Williams Landing Suite 394,Apt. 516,13056,campinas,SP,2017-01-01,2017-01-01
...,...,...,...,...,...,...,...,...,...
99436,c22ccad6-4ac5-479a-acdd-8a08205c5d0f,17ddf5dd5d51696bb3d7c6291687be6f,2806 Chris Dam Apt. 954,Apt. 931,3937,sao paulo,SP,2017-01-01,2017-01-01
99437,3810bd1a-44ea-41b2-8f7f-65c2151c3e2c,e7b71a9017aa05c9a7fd292d714858e8,7613 Edward Spring Apt. 859,Apt. 426,6764,taboao da serra,SP,2017-01-01,2017-01-01
99438,72e14e41-ba22-4289-8578-1a95af2c8d52,5e28dfe12db7fb50a4b2f691faecea5e,590 Hamilton Camp Apt. 437,Suite 704,60115,fortaleza,CE,2017-01-01,2017-01-01
99439,19fe647f-2a9c-4862-84ac-77bb9a4a1fcd,56b18e2166679b8a959d72dd06da27f9,25971 Robert Ramp Suite 536,Suite 826,92120,canoas,RS,2017-01-01,2017-01-01


In [151]:
def build_customer_accounts(customers: pd.DataFrame) -> pd.DataFrame:
    acct = pd.DataFrame({
        "account_id": [str(uuid.uuid4()) for _ in range(len(customers))],
        "customer_id": customers["customer_id"].values,
        "account_status": np.where(np.random.rand(len(customers)) < 0.98, "ACTIVE", "SUSPENDED"),
        "created_ts": pd.Timestamp("2017-01-01"),
        "last_updated_ts": pd.Timestamp("2017-01-01")
    })
    return acct

In [152]:
customer_accounts = build_customer_accounts(customers)

In [96]:
def build_inventory(products: pd.DataFrame) -> pd.DataFrame:
    inv = pd.DataFrame({
        "product_id": products["product_id"].values,
        "on_hand_qty": np.random.randint(10, 500, size=len(products)),
        "reserved_qty": np.random.randint(0, 20, size = len(products)),
        "warehouse_id": np.random.choice(["WH1", "WH2", "WH3"], size = len(products)),
        "created_ts": pd.Timestamp("2017-01-01"),
        "last_updated_ts": pd.Timestamp("2017-01-01")
    })
    return inv

In [97]:
inventory = build_inventory(products)

In [99]:
inventory

,product_id,on_hand_qty,reserved_qty,warehouse_id,created_ts,last_updated_ts
0,1e9e8ef04dbcff4541ed26657ea517e5,124,10,WH1,2017-01-01,2017-01-01
1,3aa071139cb16b67ca9e5dea641aaa2f,57,3,WH3,2017-01-01,2017-01-01
2,96bd76ec8810374ed1b65e291975717f,336,18,WH3,2017-01-01,2017-01-01
3,cef67bcfe19066a932b7673e239eb23d,422,17,WH2,2017-01-01,2017-01-01
4,9dc1a7de274444849c219cff195d0b71,142,16,WH2,2017-01-01,2017-01-01
...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,13,1,WH3,2017-01-01,2017-01-01
32947,bf4538d88321d0fd4412a93c974510e6,164,2,WH3,2017-01-01,2017-01-01
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,398,10,WH1,2017-01-01,2017-01-01
32949,83808703fc0706a22e264b9d75f04a2e,233,9,WH2,2017-01-01,2017-01-01


In [100]:
base_orders = olist_data["orders"].copy()
base_order_items = olist_data["order_items"].copy()
base_payments = olist_data["payments"].copy()

In [113]:
def build_shipments_from_orders(orders: pd.DataFrame) -> pd.DataFrame:
    shp = pd.DataFrame({
        "shipment_id": [str(uuid.uuid4()) for _ in range(len(orders))],
        "order_id": orders["order_id"].values,
        "shipment_status": np.where(orders["order_status"].isin(["delivered"]), 
                                         "DELIVERED",
                                         np.where(orders["order_status"].isin(["shipped"]), 
                                                  "SHIPPED",
                                                  np.where(orders["order_status"].isin(["canceled"]), 
                                                           "CANCELED", 
                                                           "CREATED"))),
        "shipping_created_ts": orders["order_purchase_timestamp"].values,
        "shipped_ts": orders["order_delivered_carrier_date"].values,
        "delivered_ts": orders["order_delivered_customer_date"].values,
        "carrier": np.random.choice(["Correios", "Loggi", "Jadlog", "TotalExpress"], size=len(orders)),
        "tracking_number": [fake.bothify(text="BR###########") for _ in range(len(orders))]
    })
    shp["created_ts"] = shp["shipping_created_ts"]
    shp["last_updated_ts"] = shp[["shipping_created_ts", "shipped_ts", "delivered_ts"]].max(axis=1)
    return shp


In [114]:
base_shipments = build_shipments_from_orders(base_orders)

In [123]:
def build_returns_from_orders_reviews(orders: pd.DataFrame, reviews: pd.DataFrame, cfg: GenConfig) -> pd.DataFrame:
    delivered = orders[orders["order_status"] == "delivered"].copy()
    delivered = delivered.merge(reviews[["order_id", "review_score"]], on="order_id", how = "left")
    delivered["delivery_delay_days"] = (
             (delivered["order_delivered_customer_date"] - delivered["order_purchase_timestamp"])
        .dt.total_seconds()/86400).fillna(0)

    # probabilty boost: bad reviews + long delay
    base_p = cfg.return_rate
    score = delivered["review_score"].fillna(5)
    delay = delivered["delivery_delay_days"].clip(0,60)
    p = base_p + (5 - score)*0.01 + (delay/60) * 0.02
    p = p.clip(0, 0.25)

    mask = np.random.rand(len(delivered)) < p.values
    ret_orders = delivered[mask].copy()

    if ret_orders.empty:
        return pd.DataFrame(columns = [
            "return_id", "order_id", "return_ts", "return_reson",
            "refund_amount", "created_ts", "last_updated_ts"
        ])

    return_ts = (ret_orders["order_delivered_customer_date"].fillna(ret_orders["order_purchase_timestamp"])
                 + pd.to_timedelta(np.random.randint(1, 21, size = len(ret_orders)), unit="D"))

    returns = pd.DataFrame({
        "return_id": [str(uuid.uuid4()) for _ in range(len(ret_orders))],
        "order_id": ret_orders["order_id"].values,
        "return_ts": return_ts.values,
        "return_reason": np.random.choice(
            ["DAMAGED", "NOT_AS_DESCRIBED", "LATE_DELIVERY", "WRONG_ITEM", "CHANGED_MIND"],
            size=len(ret_orders),
            p=[0.25, 0.25, 0.20, 0.10, 0.20]
        ),
        "refund_amount": np.random.uniform(5.0, 250.0, size=len(ret_orders)).round(2),
    })
    returns["created_ts"] = returns["return_ts"]
    returns["last_updated_ts"] = returns["return_ts"]
    return returns
    

In [124]:
base_returns = build_returns_from_orders_reviews(base_orders, olist_data["reviews"], cfg)

In [154]:
# For daily snapshots, we maintain "current state" for dims/state tables
cur = {
    "customers": customers,
    "customer_addresses": customer_addresses,
    "customer_accounts": customer_accounts,
    "products": products,
    "suppliers": suppliers,
    "inventory": inventory,
}



In [155]:
# Facts accumulate day by day (append)
facts = {
    "orders": base_orders.copy(),
    "order_items": base_order_items.copy(),
    "payments": base_payments.copy(),
    "shipments": base_shipments.copy(),
    "returns": base_returns.copy(),
}

In [128]:
# Ensure created_ts/last_updated_ts in baseline facts
for t in ["orders", "order_items", "payments", "shipments", "returns"]:
    if "created_ts" not in facts[t].columns:
        facts[t]["created_ts"] = pd.Timestamp("2017-01-01")
    if "last_updated_ts" not in facts[t].columns:
        facts[t]["last_updated_ts"] = pd.Timestamp("2017-01-01")

In [133]:

def pick_ids(series: pd.Series, frac: float) -> List[str]:
    n = int(len(series) * frac)
    n = max(0, n)
    if n == 0:
        return []
    return series.sample(n=min(n, len(series)), replace=False).tolist()

In [134]:


def mutate_customers(customers: pd.DataFrame, addresses: pd.DataFrame, accounts: pd.DataFrame, cfg: GenConfig, day: date
                     ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      updated customers, addresses, accounts,
      delta_customers, delta_addresses, delta_accounts
    """
    customers = customers.copy()
    addresses = addresses.copy()
    accounts = accounts.copy()

    delta_customers = []
    delta_addresses = []
    delta_accounts = []

    # CHANGES
    change_ids = pick_ids(customers["customer_id"], cfg.customer_change_rate)
    if change_ids:
        c_mask = customers["customer_id"].isin(change_ids)
        # change city/state sometimes
        customers.loc[c_mask, "customer_city"] = [fake.city().lower() for _ in range(c_mask.sum())]
        customers.loc[c_mask, "customer_state"] = np.random.choice(["SP","RJ","MG","RS","PR","SC","BA","DF","GO"], size=c_mask.sum())
        customers.loc[c_mask, "last_updated_ts"] = pd.Timestamp(day)

        # addresses follow
        a_mask = addresses["customer_id"].isin(change_ids)
        addresses.loc[a_mask, "city"] = [fake.city().lower() for _ in range(a_mask.sum())]
        addresses.loc[a_mask, "state"] = np.random.choice(["SP","RJ","MG","RS","PR","SC","BA","DF","GO"], size=a_mask.sum())
        addresses.loc[a_mask, "last_updated_ts"] = pd.Timestamp(day)

        # account segment/status changes
        acc_mask = accounts["customer_id"].isin(change_ids)
        if acc_mask.any():
            accounts.loc[acc_mask, "account_status"] = np.random.choice(["ACTIVE", "ACTIVE", "ACTIVE", "SUSPENDED"], size=acc_mask.sum())
            accounts.loc[acc_mask, "last_updated_ts"] = pd.Timestamp(day)

        delta_customers.append(customers.loc[c_mask].assign(op="C", event_dt=str(day)))
        delta_addresses.append(addresses.loc[a_mask].assign(op="C", event_dt=str(day)))
        delta_accounts.append(accounts.loc[acc_mask].assign(op="C", event_dt=str(day)))

    # DELETES (soft: remove from snapshot; emit D in delta)
    delete_ids = pick_ids(customers["customer_id"], cfg.customer_delete_rate)
    if delete_ids:
        # emit delta rows (minimal keys is fine, but we include full record for convenience)
        d_c = customers[customers["customer_id"].isin(delete_ids)].copy()
        d_a = addresses[addresses["customer_id"].isin(delete_ids)].copy()
        d_acc = accounts[accounts["customer_id"].isin(delete_ids)].copy()

        delta_customers.append(d_c.assign(op="D", event_dt=str(day)))
        delta_addresses.append(d_a.assign(op="D", event_dt=str(day)))
        delta_accounts.append(d_acc.assign(op="D", event_dt=str(day)))

        customers = customers[~customers["customer_id"].isin(delete_ids)]
        addresses = addresses[~addresses["customer_id"].isin(delete_ids)]
        accounts = accounts[~accounts["customer_id"].isin(delete_ids)]

    # INSERTS
    ins_n = int(len(customers) * cfg.customer_insert_rate)
    if ins_n > 0:
        new_customers = []
        new_addresses = []
        new_accounts = []
        for _ in range(ins_n):
            cid = str(uuid.uuid4())
            cust_unique = str(uuid.uuid4())
            zip_prefix = random.randint(1000, 99999)
            city = fake.city().lower()
            state = random.choice(["SP","RJ","MG","RS","PR","SC","BA","DF","GO"])
            new_customers.append({
                "customer_id": cid,
                "customer_unique_id": cust_unique,
                "customer_zip_code_prefix": zip_prefix,
                "customer_city": city,
                "customer_state": state,
                "created_ts": pd.Timestamp(day),
                "last_updated_ts": pd.Timestamp(day),
            })
            aid = str(uuid.uuid4())
            new_addresses.append({
                "address_id": aid,
                "customer_id": cid,
                "address_line1": fake.street_address(),
                "address_line2": fake.secondary_address(),
                "zip_code_prefix": zip_prefix,
                "city": city,
                "state": state,
                "created_ts": pd.Timestamp(day),
                "last_updated_ts": pd.Timestamp(day),
            })
            new_accounts.append({
                "account_id": str(uuid.uuid4()),
                "customer_id": cid,
                "account_status": "ACTIVE",
                "created_ts": pd.Timestamp(day),
                "last_updated_ts": pd.Timestamp(day),
            })
        new_cdf = pd.DataFrame(new_customers)
        new_adf = pd.DataFrame(new_addresses)
        new_accdf = pd.DataFrame(new_accounts)

        customers = pd.concat([customers, new_cdf], ignore_index=True)
        addresses = pd.concat([addresses, new_adf], ignore_index=True)
        accounts = pd.concat([accounts, new_accdf], ignore_index=True)

        delta_customers.append(new_cdf.assign(op="I", event_dt=str(day)))
        delta_addresses.append(new_adf.assign(op="I", event_dt=str(day)))
        delta_accounts.append(new_accdf.assign(op="I", event_dt=str(day)))

    dc = pd.concat(delta_customers, ignore_index=True) if delta_customers else pd.DataFrame()
    da = pd.concat(delta_addresses, ignore_index=True) if delta_addresses else pd.DataFrame()
    dacc = pd.concat(delta_accounts, ignore_index=True) if delta_accounts else pd.DataFrame()

    return customers, addresses, accounts, dc, da, dacc


In [135]:

def mutate_products(products: pd.DataFrame, cfg: GenConfig, day: date
                    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    products = products.copy()
    deltas = []

    # Ensure timestamps exist
    if "created_ts" not in products.columns:
        products["created_ts"] = pd.Timestamp("2017-01-01")
    if "last_updated_ts" not in products.columns:
        products["last_updated_ts"] = pd.Timestamp("2017-01-01")

    # CHANGES
    change_ids = pick_ids(products["product_id"], cfg.product_change_rate)
    if change_ids:
        m = products["product_id"].isin(change_ids)
        # tweak dimensions slightly (realistic corrections)
        products.loc[m, "product_weight_g"] = (products.loc[m, "product_weight_g"].fillna(1000) * np.random.uniform(0.9, 1.1, size=m.sum())).round()
        products.loc[m, "product_length_cm"] = (products.loc[m, "product_length_cm"].fillna(20) * np.random.uniform(0.9, 1.1, size=m.sum())).round()
        products.loc[m, "last_updated_ts"] = pd.Timestamp(day)
        deltas.append(products.loc[m].assign(op="C", event_dt=str(day)))

    # DELETES
    delete_ids = pick_ids(products["product_id"], cfg.product_delete_rate)
    if delete_ids:
        d = products[products["product_id"].isin(delete_ids)].copy()
        deltas.append(d.assign(op="D", event_dt=str(day)))
        products = products[~products["product_id"].isin(delete_ids)]

    # INSERTS
    ins_n = int(len(products) * cfg.product_insert_rate)
    if ins_n > 0:
        new_rows = []
        for _ in range(ins_n):
            pid = str(uuid.uuid4())
            new_rows.append({
                "product_id": pid,
                "product_category_name": random.choice(products["product_category_name"].dropna().tolist()),
                "product_name_lenght": random.randint(5, 60),
                "product_description_lenght": random.randint(30, 1500),
                "product_photos_qty": random.randint(1, 8),
                "product_weight_g": random.randint(50, 5000),
                "product_length_cm": random.randint(5, 120),
                "product_height_cm": random.randint(2, 60),
                "product_width_cm": random.randint(2, 80),
                "created_ts": pd.Timestamp(day),
                "last_updated_ts": pd.Timestamp(day),
            })
        new_df = pd.DataFrame(new_rows)
        products = pd.concat([products, new_df], ignore_index=True)
        deltas.append(new_df.assign(op="I", event_dt=str(day)))

    dp = pd.concat(deltas, ignore_index=True) if deltas else pd.DataFrame()
    return products, dp


In [159]:

def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def write_table(df: pd.DataFrame, path: str, fmt: str = "csv") -> None:
    ensure_dir(path)
    file_path = os.path.join(path, "part-00000.csv")
    df.to_csv(file_path, index=False)


def dt_path(root: str, kind: str, dt: date, table: str) -> str:
    # kind: "snapshots" or "deltas"
    return os.path.join(root, "raw", kind, f"dt={dt.isoformat()}", table)
    

In [161]:
root = cfg.out_dir
ensure_dir(root)

for i in range(cfg.num_days):
    day = cfg.start_date + timedelta(days=i)

    # ---- DIMS: mutate and produce deltas with op ----
    cur["customers"], cur["customer_addresses"], cur["customer_accounts"], dc, da, dacc = mutate_customers(
        cur["customers"], cur["customer_addresses"], cur["customer_accounts"], cfg, day
    )
    cur["products"], dp = mutate_products(cur["products"], cfg, day)

    # Suppliers: optional small changes (keep stable for now)
    dsup = pd.DataFrame()  # keep empty

    # ---- FACTS: generate new daily facts (append-only) ----
    new_orders, new_items, new_payments, new_shipments = generate_new_orders(
        customers=cur["customers"],
        products=cur["products"],
        sellers=olist_data["sellers"],
        cfg=cfg,
        day=day
    )
    new_returns = generate_returns_for_new_delivered(new_shipments, new_payments, cfg, day)

    facts["orders"] = pd.concat([facts["orders"], new_orders], ignore_index=True)
    facts["order_items"] = pd.concat([facts["order_items"], new_items], ignore_index=True)
    facts["payments"] = pd.concat([facts["payments"], new_payments], ignore_index=True)
    facts["shipments"] = pd.concat([facts["shipments"], new_shipments], ignore_index=True)
    if not new_returns.empty:
        facts["returns"] = pd.concat([facts["returns"], new_returns], ignore_index=True)

    # ---- INVENTORY: update based on sales ----
    cur["inventory"], dinv = update_inventory_for_day(cur["inventory"], new_items, cfg, day)

    # ---- WRITE DAILY SNAPSHOTS (full dumps) ----
    # Dims/state snapshots
    for tbl in ["customers", "customer_addresses", "customer_accounts", "products", "suppliers", "inventory"]:
        out = dt_path(root, "snapshots", day, tbl)
        write_table(cur[tbl], out, cfg.file_format)

    # Facts snapshots (optional full dump each day; enterprises sometimes do it)
    # For your interview practice, it's useful to have daily full snapshots too.
    for tbl in ["orders", "order_items", "payments", "shipments", "returns"]:
        out = dt_path(root, "snapshots", day, tbl)
        write_table(facts[tbl], out, cfg.file_format)

    # ---- WRITE DAILY DELTAS (with op) ----
    deltas: Dict[str, pd.DataFrame] = {
        "customers": dc,
        "customer_addresses": da,
        "customer_accounts": dacc,
        "products": dp,
        "suppliers": dsup,
        "inventory": dinv,
        # Facts: treat all new rows as inserts
        "orders": new_orders.assign(op="I", event_dt=str(day)),
        "order_items": new_items.assign(op="I", event_dt=str(day)),
        "payments": new_payments.assign(op="I", event_dt=str(day)),
        "shipments": new_shipments.assign(op="I", event_dt=str(day)),
        "returns": (new_returns.assign(op="I", event_dt=str(day)) if not new_returns.empty else pd.DataFrame()),
    }

    for tbl, df in deltas.items():
        out = dt_path(root, "deltas", day, tbl)
        if df is None or df.empty:
            # still create folder so pipelines don't break
            ensure_dir(out)
            # write an empty file with headers when possible
            if tbl in cur:
                empty = cur[tbl].head(0).copy()
            elif tbl in facts:
                empty = facts[tbl].head(0).copy()
            else:
                empty = pd.DataFrame()
            if "op" not in empty.columns:
                empty["op"] = pd.Series(dtype="string")
            if "event_dt" not in empty.columns:
                empty["event_dt"] = pd.Series(dtype="string")
            write_table(empty, out, cfg.file_format)
        else:
            write_table(df, out, cfg.file_format)

    print(f"Generated day {day} snapshots + deltas")

print(f"\nDone. Output at: {os.path.abspath(root)}")
print("Example paths:")
print(f"  {root}/raw/snapshots/dt={cfg.start_date.isoformat()}/customers/part-00000.csv")
print(f"  {root}/raw/deltas/dt={cfg.start_date.isoformat()}/customers/part-00000.csv")


Generated day 2018-01-01 snapshots + deltas
Generated day 2018-01-02 snapshots + deltas
Generated day 2018-01-03 snapshots + deltas
Generated day 2018-01-04 snapshots + deltas
Generated day 2018-01-05 snapshots + deltas
Generated day 2018-01-06 snapshots + deltas
Generated day 2018-01-07 snapshots + deltas
Generated day 2018-01-08 snapshots + deltas
Generated day 2018-01-09 snapshots + deltas
Generated day 2018-01-10 snapshots + deltas
Generated day 2018-01-11 snapshots + deltas
Generated day 2018-01-12 snapshots + deltas
Generated day 2018-01-13 snapshots + deltas
Generated day 2018-01-14 snapshots + deltas

Done. Output at: /home/jovyan/work/Data Engineering/1. enterprise-grade batch analytics warehouse/output
Example paths:
  output/raw/snapshots/dt=2018-01-01/customers/part-00000.csv
  output/raw/deltas/dt=2018-01-01/customers/part-00000.csv


In [165]:
base_orders[["order_delivered_customer_date", "order_delivered_carrier_date", "order_estimated_delivery_date"]].max()

order_delivered_customer_date   2018-10-17 13:22:46
order_delivered_carrier_date    2018-09-11 19:48:28
order_estimated_delivery_date   2018-11-12 00:00:00
dtype: datetime64[us]

# create s3 files.

In [195]:
import boto3
import json
import awswrangler as wr

client = boto3.client(service_name='secretsmanager')
secret = client.get_secret_value(SecretId='test/secretsmanager')
secret_dict = json.loads(secret['SecretString'])
s3_bucket = secret_dict['s3_bucket']


def dt_path_s3(root: str, kind: str, dt: date, table: str) -> str:
    # kind: "snapshots" or "deltas"
    return os.path.join(root, "raw", kind, table, f"{dt.isoformat()}")


def write_table_s3(df:pd.DataFrame, path):
    s3_path = f's3://{s3_bucket}/{path}/file.csv'
    wr.s3.to_csv(df=df, path=s3_path, index=False)

In [199]:
root = cfg.out_dir

for i in range(cfg.num_days):
    day = cfg.start_date + timedelta(days=i)

    # ---- DIMS: mutate and produce deltas with op ----
    cur["customers"], cur["customer_addresses"], cur["customer_accounts"], dc, da, dacc = mutate_customers(
        cur["customers"], cur["customer_addresses"], cur["customer_accounts"], cfg, day
    )
    cur["products"], dp = mutate_products(cur["products"], cfg, day)

    # Suppliers: optional small changes (keep stable for now)
    dsup = pd.DataFrame()  # keep empty

    # ---- FACTS: generate new daily facts (append-only) ----
    new_orders, new_items, new_payments, new_shipments = generate_new_orders(
        customers=cur["customers"],
        products=cur["products"],
        sellers=olist_data["sellers"],
        cfg=cfg,
        day=day
    )
    new_returns = generate_returns_for_new_delivered(new_shipments, new_payments, cfg, day)

    facts["orders"] = pd.concat([facts["orders"], new_orders], ignore_index=True)
    facts["order_items"] = pd.concat([facts["order_items"], new_items], ignore_index=True)
    facts["payments"] = pd.concat([facts["payments"], new_payments], ignore_index=True)
    facts["shipments"] = pd.concat([facts["shipments"], new_shipments], ignore_index=True)
    if not new_returns.empty:
        facts["returns"] = pd.concat([facts["returns"], new_returns], ignore_index=True)

    # ---- INVENTORY: update based on sales ----
    cur["inventory"], dinv = update_inventory_for_day(cur["inventory"], new_items, cfg, day)

    # ---- WRITE DAILY SNAPSHOTS (full dumps) ----
    # Dims/state snapshots
    for tbl in ["customers", "customer_addresses", "customer_accounts", "products", "suppliers", "inventory"]:
        out = dt_path_s3(root, "snapshots", day, tbl)
        write_table_s3(cur[tbl], out)

    # Facts snapshots (optional full dump each day; enterprises sometimes do it)
    # For your interview practice, it's useful to have daily full snapshots too.
    for tbl in ["orders", "order_items", "payments", "shipments", "returns"]:
        out = dt_path_s3(root, "snapshots", day, tbl)
        write_table_s3(facts[tbl], out)

    # ---- WRITE DAILY DELTAS (with op) ----
    deltas: Dict[str, pd.DataFrame] = {
        "customers": dc,
        "customer_addresses": da,
        "customer_accounts": dacc,
        "products": dp,
        "suppliers": dsup,
        "inventory": dinv,
        # Facts: treat all new rows as inserts
        "orders": new_orders.assign(op="I", event_dt=str(day)),
        "order_items": new_items.assign(op="I", event_dt=str(day)),
        "payments": new_payments.assign(op="I", event_dt=str(day)),
        "shipments": new_shipments.assign(op="I", event_dt=str(day)),
        "returns": (new_returns.assign(op="I", event_dt=str(day)) if not new_returns.empty else pd.DataFrame()),
    }

    for tbl, df in deltas.items():
        out = dt_path_s3(root, "deltas", day, tbl)
        if df is None or df.empty:
            # still create folder so pipelines don't break
            ensure_dir(out)
            # write an empty file with headers when possible
            if tbl in cur:
                empty = cur[tbl].head(0).copy()
            elif tbl in facts:
                empty = facts[tbl].head(0).copy()
            else:
                empty = pd.DataFrame()
            if "op" not in empty.columns:
                empty["op"] = pd.Series(dtype="string")
            if "event_dt" not in empty.columns:
                empty["event_dt"] = pd.Series(dtype="string")
            write_table_s3(empty, out)
        else:
            write_table_s3(df, out)

    print(f"Generated day {day} snapshots + deltas")

print(f"\nDone. Output at: {os.path.abspath(root)}")
print("Example paths:")
print(f"  {root}/raw/snapshots/customers/dt={cfg.start_date.isoformat()}/part-00000.csv")
print(f"  {root}/raw/deltas/customers/dt={cfg.start_date.isoformat()}/part-00000.csv")


Generated day 2019-01-01 snapshots + deltas
Generated day 2019-01-02 snapshots + deltas
Generated day 2019-01-03 snapshots + deltas
Generated day 2019-01-04 snapshots + deltas
Generated day 2019-01-05 snapshots + deltas
Generated day 2019-01-06 snapshots + deltas
Generated day 2019-01-07 snapshots + deltas
Generated day 2019-01-08 snapshots + deltas
Generated day 2019-01-09 snapshots + deltas
Generated day 2019-01-10 snapshots + deltas
Generated day 2019-01-11 snapshots + deltas
Generated day 2019-01-12 snapshots + deltas
Generated day 2019-01-13 snapshots + deltas
Generated day 2019-01-14 snapshots + deltas
Generated day 2019-01-15 snapshots + deltas

Done. Output at: /home/jovyan/work/Data Engineering/1. enterprise-grade batch analytics warehouse/output
Example paths:
  output/raw/snapshots/customers/dt=2019-01-01/part-00000.csv
  output/raw/deltas/customers/dt=2019-01-01/part-00000.csv
